In [1]:
import os
import sys

os.chdir("..")
sys.path.append("..")

In [2]:
import yaml

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from bokeh.io import output_notebook, show
from bokeh.io.export import export_svg
from bokeh.layouts import row
from bokeh.plotting import figure
from bokeh.transform import log_cmap, linear_cmap
from bokeh.util.hex import hexbin, cartesian_to_axial

from gluonts.dataset.repository.datasets import get_dataset
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.seasonal import STL
from tqdm import tqdm

from src.utils.data_loading import load_features, load_score, load_test_data, load_train_data

/anaconda/envs/whatif/lib/python3.8/site-packages/gluonts/json.py:45: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(


In [3]:
def create_color_arr(scores, quantiles=None):
    if quantiles is None:
        quantiles = np.nanquantile(scores, [0.25, 0.75])
    
    low = scores < quantiles[0]
    high = scores > quantiles[1]
    medium = np.logical_and(~low, ~high)
    
    colors = np.empty_like(scores)
    colors[low] = 0
    colors[medium] = 1
    colors[high] = 2
    
    return colors


def color_bin(bins, pca_data, colors, column_name):
    q, r = cartesian_to_axial(pca_data[:, 0], pca_data[:, 1], 0.1, "pointytop")
    df = pd.DataFrame(dict(r=r, q=q))
    groups = df.groupby(["q", "r"])
    
    for (q, r), indexes in groups.groups.items():
        color = np.nanmean(colors[indexes])
        bins.loc[(bins["q"] == q) & (bins["r"] == r), column_name] = color
    
    return bins


def get_fig(title, xrange, yrange):
    p = figure(title=title, tools="", match_aspect=True, x_range=xrange, y_range=yrange)
    p.output_backend = "svg"
    p.title.align = "center"
    p.grid.visible = False
    return p


def create_and_plot_hexbin(original_scores, new_scores, pca_data, figdir, dataset, model, metric):
    # create a seperate folder for each dataset and model
    figdir = os.path.join(figdir, dataset, model)
    if not os.path.isdir(figdir):
        os.makedirs(figdir, exist_ok=True)
    
    # create hexbins
    bins = hexbin(pca_data[:, 0], pca_data[:, 1], 0.1)
    bins["original_colors"] = np.nan
    bins["new_colors"] = np.nan
    
    orig_quantiles = np.nanquantile(original_scores, [0.25, 0.75])
    original_colors = create_color_arr(original_scores, orig_quantiles)
    new_colors = create_color_arr(new_scores, orig_quantiles)

    bins = color_bin(bins, pca_data, original_colors, "original_colors")
    bins = color_bin(bins, pca_data, new_colors, "new_colors")
    
    # plot original model
    p1 = get_fig(f"{model} trained with reduced training data", None, None)
    p1.hex_tile(q="q", r="r", size=0.1, line_color=None, source=bins,
                fill_color=linear_cmap("original_colors", "Viridis256", min(bins.original_colors), max(bins.original_colors)))
    
    # plot new model
    p2 = get_fig(f"{model} trained with reduced training data", None, None)
    p2.hex_tile(q="q", r="r", size=0.1, line_color=None, source=bins,
                fill_color=linear_cmap("new_colors", "Viridis256", min(bins.new_colors), max(bins.new_colors)))
    
    p = row([p1, p2])
    export_svg(p, filename=os.path.join(figdir, f"{dataset}_{model}_real_hold_out_{metric}_hexbin.svg"))

In [4]:
models = ["feedforward", "nbeats_g", "seq2seq", "tcn", "transformer"]
names = ["Fully-connected", "N-BEATS", "LSTM", "TCN", "Transformer"]
dataset = "m4_monthly"
metric = "mae"

datadir = f"data/{dataset}"
figdir = "./figures/OOD"
if not os.path.isdir(figdir):
    os.makedirs(figdir, exist_ok=True)

In [5]:
train_features = load_features(datadir, train=True)
test_features = load_features(datadir, train=False)

scaler = StandardScaler()
norm_train_features = scaler.fit_transform(train_features)
norm_test_features = scaler.transform(test_features)

pca = PCA(n_components=2)
train_pca_data = pca.fit_transform(norm_train_features)
test_pca_data = pca.transform(norm_test_features)

In [6]:
ood_mask1 = test_pca_data[:, 0] < 0
ood_mask2 = test_pca_data[:, 1] < 2

ood_mask = np.logical_and(ood_mask1, ood_mask2)

In [7]:
orig_mask = test_pca_data[:, 0] > 0

In [9]:
for model, name in zip(models, names):
    print(name)
    experiment_dir = f"experiments/{dataset}/{model}_gen_real_hold_out" 
    
    original_scores = load_score(experiment_dir, metric)
    augmented_scores = load_score(experiment_dir + "_aug", metric)
    
    create_and_plot_hexbin(original_scores, augmented_scores, test_pca_data, figdir, dataset, name, metric)
    
    filtered_original_scores = np.nanmean(original_scores[ood_mask])
    filtered_augmented_scores = np.nanmean(augmented_scores[ood_mask])
    
    filtered_all_percentage = np.round((np.abs(np.nanmean(original_scores) - np.nanmean(augmented_scores)) / np.nanmean(original_scores)) * 100, 3)
    filtered_original_percentage = np.round((np.abs(np.nanmean(original_scores[orig_mask]) - np.nanmean(augmented_scores[orig_mask])) / np.nanmean(original_scores[orig_mask])) * 100, 3)
    filtered_ood_percentage = np.round((np.abs(np.nanmean(original_scores[ood_mask]) - np.nanmean(augmented_scores[ood_mask])) / np.nanmean(original_scores[ood_mask])) * 100, 3)
    filtered_not_ood_percentage = np.round((np.abs(np.nanmean(original_scores[~ood_mask]) - np.nanmean(augmented_scores[~ood_mask])) / np.nanmean(original_scores[~ood_mask])) * 100, 3)
    
    filtered_all_str = "+" + str(filtered_all_percentage) if np.nanmean(augmented_scores) >= np.nanmean(original_scores) else "-" + str(filtered_all_percentage)
    filtered_original_str = "+" + str(filtered_original_percentage) if np.nanmean(augmented_scores[orig_mask]) >= np.nanmean(original_scores[orig_mask]) else "-" + str(filtered_original_percentage)
    filtered_ood_str = "+" + str(filtered_ood_percentage) if np.nanmean(augmented_scores[ood_mask]) >= np.nanmean(original_scores[ood_mask]) else "-" + str(filtered_ood_percentage)
    filtered_not_ood_str ="+" + str(filtered_not_ood_percentage) if np.nanmean(augmented_scores[~ood_mask]) >= np.nanmean(original_scores[~ood_mask]) else "-" + str(filtered_not_ood_percentage)
    
    print(f"\tOriginal model average {metric} on full test data: {np.round(np.nanmean(original_scores), 3)}")
    print(f"\tOriginal model average {metric} on in-distribution test data: {np.round(np.nanmean(original_scores[orig_mask]), 3)}")
    print(f"\tOriginal model average {metric} on out-of-distribution test data: {np.round(filtered_original_scores, 3)}")
    print(f"\tAugmented model average {metric} on full test data: {np.round(np.nanmean(augmented_scores), 3)}")
    print(f"\tAugmented model average {metric} on in-distribution test data: {np.round(np.nanmean(augmented_scores[orig_mask]), 3)}")
    print(f"\tAugmented model average {metric} on out-of-distribution test data: {np.round(filtered_augmented_scores, 3)}")
    print()
    print(f"\tPercentage change on all test data: {filtered_all_str}")
    print(f"\tPercentage change on in-distribution test data: {filtered_original_str}")
    print(f"\tPercentage change on out-of-distribution data: {filtered_ood_str}")
    print(f"\tPercentage change on not out-of-distribution data: {filtered_not_ood_str}")

Fully-connected
	Original model average mae on full test data: 656.486
	Original model average mae on in-distribution test data: 713.781
	Original model average mae on out-of-distribution test data: 528.009
	Augmented model average mae on full test data: 650.551
	Augmented model average mae on in-distribution test data: 776.41
	Augmented model average mae on out-of-distribution test data: 456.101

	Percentage change on all test data: -0.904
	Percentage change on in-distribution test data: +8.774
	Percentage change on out-of-distribution data: -13.619
	Percentage change on not out-of-distribution data: +6.52
N-BEATS
	Original model average mae on full test data: 615.116
	Original model average mae on in-distribution test data: 662.597
	Original model average mae on out-of-distribution test data: 490.612
	Augmented model average mae on full test data: 609.246
	Augmented model average mae on in-distribution test data: 703.695
	Augmented model average mae on out-of-distribution test data: 